In [ ]:
import numpy as np
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')
from glob import glob
from tqdm import tqdm
import apt
from bs4 import BeautifulSoup
folder = ''
api = ''

In [ ]:
!pip install PyPDF2 wget pypdf tabula-py pdfminer camelot-py[cv] -q
import tabula, camelot
import pypdf
import PyPDF2
import wget
from pypdf import PdfReader

### download table of drugs

In [ ]:
data_f = folder + 'data'
url = 'https://www.ema.europa.eu/sites/default/files/Medicines_output_european_public_assessment_reports.xlsx'
wget.download(url, out = data_f)

'/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/eu_drug_label/data/Medicines_output_european_public_assessment_reports (1).xlsx'

### download individual drug files

In [ ]:
#download the table of all drug labels and their links first. read this in, and filter for human drugs
df = pd.read_excel(folder+'data/Medicines_output_european_public_assessment_reports (1).xlsx', skiprows=8)
df = df[df.Category == 'Human']
print(df.shape[0])
df.head(1)

1741


,Category,Medicine name,Therapeutic area,International non-proprietary name (INN) / common name,Active substance,Product number,Patient safety,Authorisation status,ATC code,Additional monitoring,...,Vet pharmacotherapeutic group,Date of opinion,Decision date,Revision number,Condition / indication,Species,ATCvet code,First published,Revision date,URL
0,Human,Steglujan,"Diabetes Mellitus, Type 2","ertugliflozin, sitagliptin","ertugliflozin l-pyroglutamic acid, sitagliptin...",EMEA/H/C/004313,no,Authorised,A10BD24,no,...,NaN,NaT,2023-09-18,11.0,Steglujan is indicated in adults aged 18 years...,NaN,NaN,2018-08-23 12:07:00,2023-09-19 12:42:00,https://www.ema.europa.eu/en/medicines/human/E...


In [ ]:
not_found = []
for i, row in tqdm(df.iterrows()):
  med = row['Medicine name']
  url = row['URL'].split('/')[-1]
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(url)
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found.append(med)

1741it [14:18,  2.03it/s]


In [ ]:
print('downloaded : {dl}, failed : {fail}, total : {tot}'.format(dl=str(df.shape[0]-len(not_found)), fail=len(not_found), tot=str(df.shape[0])))
print('retry download')

downloaded : 1564, failed : 177, total : 1741
retry download


In [ ]:
not_found = []
for med in tqdm(df['Medicine name']):
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(str(med).lower())
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found.append(med)

100%|██████████| 1741/1741 [14:23<00:00,  2.02it/s]


In [ ]:
print('downloaded : {dl}, failed : {fail}, total : {tot}'.format(dl=str(df.shape[0]-len(not_found)), fail=len(not_found), tot=str(df.shape[0])))
print('retry download')

downloaded : 1564, failed : 177, total : 1741
retry download


In [ ]:
not_found_2 = []
for med in tqdm(not_found):
  if 'known as' in med:
    med_alt = med.split(' (')[0].replace(' ','-').lower()
  else:
    med_alt = med.replace(' ','-').replace('(','').replace(')','').lower()
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(med_alt)
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found_2.append(med)

100%|██████████| 537/537 [03:34<00:00,  2.50it/s]


In [ ]:
len(not_found_2)

193

In [ ]:
not_found_3 = []
for med in tqdm(not_found_2):
  if 'previously' in med:
    med_alt = med.split(' (')[0].replace(' ','-').lower()
  else:
    med_alt = med.replace(' ','-').replace('(','').replace(')','').lower()
  url = 'https://www.ema.europa.eu/en/documents/product-information/{}-epar-product-information_en.pdf'.format(med_alt)
  try:
    wget.download(url, out = folder+'data/raw/{}_label.pdf'.format(str(med).lower()))
  except:
    not_found_3.append(med)

100%|██████████| 193/193 [01:18<00:00,  2.44it/s]


In [ ]:
df[df['Medicine name'].isin(not_found_3)]['Authorisation status'].value_counts()

Withdrawn     62
Refused       48
Authorised    38
Name: Authorisation status, dtype: int64

In [ ]:
a = df[df['Medicine name'].isin(not_found_3)]
a[a['Authorisation status'] == 'Authorised'].URL.tolist()[0]

'https://www.ema.europa.eu/en/medicines/human/EPAR/amlodipine-valsartan-mylan'

### extract pdf text into json

In [ ]:
!mkdir $folder'data/raw_txt'
!mkdir $folder'data/raw_tbl'

In [ ]:
pdfs = glob(folder+'data/raw/*_label.pdf')
len(pdfs)

1604

In [ ]:
for pdf in tqdm(pdfs[0:]):
  drug = pdf.split('/')[-1].split('_label.pdf')[0]
  pdf_file = open(pdf, 'rb')
  read_pdf = PyPDF2.PdfReader(pdf_file)
  n_pages = len(read_pdf.pages)
  p_text = ''
  for n in range(n_pages):
    page = read_pdf.pages[n]
    page_content = page.extract_text()
    p_text += page_content
  txt_drug = folder+'data/raw_txt/{}.txt'.format(drug)
  with open(txt_drug, 'w+') as f:
    f.write(p_text)

100%|██████████| 58/58 [02:49<00:00,  2.92s/it]


In [ ]:
files = glob(folder+'data/raw/*.pdf')
for i, f in tqdm(enumerate(pdfs[0:])): 
  try:
    tables = tabula.read_pdf(f,pages='all', silent=True)
    for i, table in enumerate(tables):
      table.to_csv(folder+'data/raw_tbl/{d}_{i}.csv'.format(d = f.split('/')[-1].split('_')[0], i = str(i)), index=False)
  except:
    continue

904it [3:05:17, 12.30s/it]


In [19]:
txts = glob(folder+'data/raw_txt/*')
tbls = glob(folder+'data/raw_tbl/*')
len(txts), len(tbls)

(1603, 138910)

In [20]:
len(list(set([i.split('/')[-1].split('_')[0] for i in tbls])))

1592